<a href="https://colab.research.google.com/github/snfruby/CS114.K21/blob/master/POS_Tagging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import thư viện cần thiết

In [34]:
pip install sklearn_crfsuite 

In [35]:
import nltk
from sklearn.model_selection import train_test_split
import re
from sklearn_crfsuite import metrics
from sklearn_crfsuite import scorers
from sklearn_crfsuite import CRF

# Upload + tiền xử lý 40 câu test


In [36]:
f = open('/content/50_60CAU.txt', 'r')
raw = f.read()

In [37]:
raw = raw.replace("\n", " ")

In [38]:
raw2 = raw.split('. ')

In [39]:
for i in range(len(raw2)):
  raw2[i] = re.sub('([.,!?()])', r' \1 ', raw2[i])
  raw2[i] = re.sub('\s{2,}', ' ', raw2[i])
raw2

['Insects , birds and fish tend to be the creatures that humans feel furthest from',
 'Unlike many mammals they do not engage in human-like behaviour',
 "The way they swarm or flock together does not usually get good press coverage either: marching like worker ants might be a common simile for city commuters , but it's a damning , not positive , image",
 'Yet a new school of scientific theory suggests that these swarms might have a lot to teach us',
 "American author Peter Miller explains , 'I used to think that individual ants knew where they were going , and what they were supposed to do when they got there",
 'But Deborah Gordon , a biologist at Stanford University , showed me that nothing an ant does makes any sense except in terms of the whole colony',
 "Which makes you wonder if , as individuals , we don't serve a similar function for the companies where we work or the communities where we live . ' Ants are not intelligent by themselves",
 'Yet as a colony , they make wise decisi

#Download Train Dataset (Treebank)

In [40]:
nltk.download('treebank')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

In [41]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [42]:
tagged_sentences = nltk.corpus.treebank.tagged_sents()

In [43]:
tagged_words=[tup for sent in tagged_sentences for tup in sent]
print("Total Number of Tagged words", len(tagged_words))
vocab=set([word for word,tag in tagged_words])
print("Vocabulary of the Corpus",len(vocab))
tags=set([tag for word,tag in tagged_words])
print("Number of Tags in the Corpus ",len(tags))

Total Number of Tagged words 100676
Vocabulary of the Corpus 12408
Number of Tags in the Corpus  46


In [44]:
print(tags)

{'-RRB-', 'VBN', 'JJ', 'WP', 'SYM', 'RBS', 'PRP', 'PDT', '#', 'JJS', '``', 'RBR', ',', 'VB', 'VBP', 'IN', 'MD', 'VBD', "''", 'NNP', '$', 'NN', 'EX', 'NNPS', 'WRB', 'PRP$', 'CD', 'WDT', 'VBZ', 'LS', 'VBG', 'NNS', '-LRB-', 'FW', 'JJR', ':', 'UH', 'DT', 'RB', 'WP$', 'CC', 'TO', 'POS', '-NONE-', '.', 'RP'}


In [45]:
train_set, test_set = train_test_split(tagged_sentences,test_size=0.2,random_state=1234)
print("Number of Sentences in Training Data ",len(train_set))
print("Number of Sentences in Testing Data ",len(test_set))

Number of Sentences in Training Data  3131
Number of Sentences in Testing Data  783


# Khởi tạo Dictionary để trích xuất đặc trưng

In [46]:
def features(sentence,index):
    ### sentence is of the form [w1,w2,w3,..], index is the position of the word in the sentence
    return {
        'is_first_capital':int(sentence[index][0].isupper()),
        'is_first_word': int(index==0),
        'is_last_word':int(index==len(sentence)-1),
        'is_complete_capital': int(sentence[index].upper()==sentence[index]),
        'prev_word':'' if index==0 else sentence[index-1],
        'next_word':'' if index==len(sentence)-1 else sentence[index+1],
        'is_numeric':int(sentence[index].isdigit()),
        'is_alphanumeric': int(bool((re.match('^(?=.*[0-9]$)(?=.*[a-zA-Z])',sentence[index])))),
        'prefix_1':sentence[index][0],
        'prefix_2': sentence[index][:2],
        'prefix_3':sentence[index][:3],
        'prefix_4':sentence[index][:4],
        'suffix_1':sentence[index][-1],
        'suffix_2':sentence[index][-2:],
        'suffix_3':sentence[index][-3:],
        'suffix_4':sentence[index][-4:],
        'word_has_hyphen': 1 if '-' in sentence[index] else 0,
        'capitals_inside': 1 if sentence[index][1:].lower() != sentence[index][1:] else 0,
         }
def untag(sentence):
    return [word for word,tag in sentence]
 
 
def prepareData(tagged_sentences):
    X,y=[],[]
    for sentences in tagged_sentences:
        X.append([features(untag(sentences), index) for index in range(len(sentences))])
        y.append([tag for word,tag in sentences])
    return X,y
X_train,y_train=prepareData(train_set)
X_test,y_test=prepareData(test_set)

# Xây dựng mô hình CRFs


In [47]:
crf = CRF(
    algorithm='lbfgs',
    c1=0.01,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.01, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

# Đánh giá mô hình Conditional Random Fields

In [48]:
y_pred=crf.predict(X_test)
print("F1 score on Test Data ")
print(metrics.flat_f1_score(y_test, y_pred,average='weighted',labels=crf.classes_))
print("F score on Training Data ")
y_pred_train=crf.predict(X_train)
metrics.flat_f1_score(y_train, y_pred_train,average='weighted',labels=crf.classes_)
 
### Look at class wise score
print(metrics.flat_classification_report(
    y_test, y_pred, labels=crf.classes_, digits=3
))


F1 score on Test Data 
0.9655175201023506
F score on Training Data 


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


              precision    recall  f1-score   support

          IN      0.979     0.983     0.981      1869
         NNP      0.961     0.986     0.973      1824
         NNS      0.965     0.982     0.973      1194
          CC      0.994     0.994     0.994       480
         VBP      0.898     0.869     0.883       283
          JJ      0.907     0.889     0.898      1147
          NN      0.947     0.948     0.948      2536
           ,      0.999     1.000     0.999       993
      -NONE-      1.000     1.000     1.000      1274
         VBG      0.920     0.926     0.923       285
          CD      0.993     0.993     0.993       671
          DT      0.992     0.994     0.993      1596
          RB      0.946     0.926     0.936       513
         WRB      1.000     1.000     1.000        40
         PRP      0.992     0.995     0.993       364
           .      1.000     1.000     1.000       777
         VBD      0.923     0.936     0.929       563
         VBN      0.899    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Đưa ra dự đoán cho tập Test set

In [49]:
def pos_tag(sentence):
    sentence_features = [features(sentence, index) for index in range(len(sentence))]
    return list(zip(sentence, crf.predict([sentence_features])[0]))

In [50]:
pos_pred = []  # list chứa kết quả dự đoán bằng model CRFs
for i in raw2:
  i = i.strip()
  sentence = i.split(" ")
  ans = pos_tag(sentence)
  pos_pred.append(ans)

In [51]:
pos = []        # list chứa kết quả dự đoán bằng thư viện nltk
for i in raw2:
  i = i.strip()
  sentence = i.split(" ")
  ans = nltk.pos_tag(sentence) 
  pos.append(ans)

In [52]:
count =[]  # kiểm tra sự khác nhau giữa 2 kế quả
for i in range(len(pos)):
  for y in range(len(pos[i])):
    if pos[i][y] != pos_pred[i][y]:
      count.append([pos[i][y], pos_pred[i][y]])
      

In [53]:
len(count)

67

In [54]:
a = raw2[2]  # in ra sự khác biệt giữa 2 câu
a= a.strip()
a= a.split(" ")
ans1 = pos_tag(a)
ans2 = nltk.pos_tag(a)

In [55]:
print(ans1,'\n',ans2)

[('The', 'DT'), ('way', 'NN'), ('they', 'PRP'), ('swarm', 'VBP'), ('or', 'CC'), ('flock', 'NN'), ('together', 'NN'), ('does', 'VBZ'), ('not', 'RB'), ('usually', 'RB'), ('get', 'VB'), ('good', 'JJ'), ('press', 'NN'), ('coverage', 'NN'), ('either:', 'CC'), ('marching', 'VBG'), ('like', 'IN'), ('worker', 'NN'), ('ants', 'NNS'), ('might', 'MD'), ('be', 'VB'), ('a', 'DT'), ('common', 'JJ'), ('simile', 'NN'), ('for', 'IN'), ('city', 'NN'), ('commuters', 'NNS'), (',', ','), ('but', 'CC'), ("it's", 'VBZ'), ('a', 'DT'), ('damning', 'NN'), (',', ','), ('not', 'RB'), ('positive', 'JJ'), (',', ','), ('image', 'NN')] 
 [('The', 'DT'), ('way', 'NN'), ('they', 'PRP'), ('swarm', 'VBP'), ('or', 'CC'), ('flock', 'VBP'), ('together', 'RB'), ('does', 'VBZ'), ('not', 'RB'), ('usually', 'RB'), ('get', 'VB'), ('good', 'JJ'), ('press', 'NN'), ('coverage', 'NN'), ('either:', 'VBD'), ('marching', 'VBG'), ('like', 'IN'), ('worker', 'NN'), ('ants', 'NNS'), ('might', 'MD'), ('be', 'VB'), ('a', 'DT'), ('common', 'J

# Predict một câu bất kì và so sánh kết quả với thư viện nltk

In [64]:
kq = "Quartz is your ally and resource in developing better ways of doing business."
kq = re.sub('([.,!?()])', r' \1 ', kq)
kq = re.sub('\s{2,}', ' ', kq)
print(pos_tag(kq.split(" ")))
print(nltk.pos_tag(kq.split(" ")))

[('Quartz', 'NNP'), ('is', 'VBZ'), ('your', 'PRP$'), ('ally', 'RB'), ('and', 'CC'), ('resource', 'NN'), ('in', 'IN'), ('developing', 'VBG'), ('better', 'JJR'), ('ways', 'NNS'), ('of', 'IN'), ('doing', 'VBG'), ('business.', 'NN')]
[('Quartz', 'NNP'), ('is', 'VBZ'), ('your', 'PRP$'), ('ally', 'NN'), ('and', 'CC'), ('resource', 'NN'), ('in', 'IN'), ('developing', 'VBG'), ('better', 'JJR'), ('ways', 'NNS'), ('of', 'IN'), ('doing', 'VBG'), ('business.', 'NN')]
